In [41]:
from tqdm import tqdm
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import torch.utils.data as data
import torchvision.transforms as transforms
import torch.nn.functional as F

import medmnist
from medmnist import BreastMNIST
from medmnist import INFO, Evaluator

import pennylane as qml

In [42]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
if torch.cuda.is_available():
    print(f"Using: {torch.cuda.get_device_name(0)}")
    print(f"CUDA: {torch.version.cuda}")
else:
    print("CUDA is not available. Using CPU.")

Using: NVIDIA GeForce RTX 4070 Ti SUPER
CUDA: 12.4


In [43]:
info = INFO['breastmnist']
data_flag = 'breastmnist'
DataClass = BreastMNIST

task = info['task']  
n_channels = info['n_channels']
n_classes = len(info['label'])

transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(mean=[.5], std=[.5]),
    lambda x: x.unsqueeze(0)  # Adiciona a dimensão do canal
])


In [44]:
data_train = DataClass(split='train', transform=transform, download=True)
data_test = DataClass(split='test', transform=transform, download=True)
data_eval = DataClass(split='val', transform=transform, download=True)

batch_size = 64  
dataloader_train = data.DataLoader(dataset=data_train, batch_size=batch_size, shuffle=True)
dataloader_test = data.DataLoader(dataset=data_test, batch_size=batch_size, shuffle=False)
dataloader_eval = data.DataLoader(dataset=data_eval, batch_size=batch_size, shuffle=False)


Using downloaded and verified file: /home/eflammere/.medmnist/breastmnist.npz
Using downloaded and verified file: /home/eflammere/.medmnist/breastmnist.npz
Using downloaded and verified file: /home/eflammere/.medmnist/breastmnist.npz


In [45]:
dev = qml.device("default.qubit", wires=4)
# Parâmetros aleatórios para o circuito
n_layers = 2
rand_params = np.random.uniform(high=2 * np.pi, size=(n_layers, 4))

@qml.qnode(dev, interface="torch")
def circuit(phi):
    # Codificação de 4 valores de entrada clássicos
    for j in range(4):
        qml.RY(np.pi * phi[j], wires=j)

    # Camadas quânticas aleatórias
    qml.templates.layers.RandomLayers(rand_params, wires=list(range(4)))

    # Medidas produzindo 4 valores clássicos de saída
    return [qml.expval(qml.PauliZ(j)) for j in range(4)]

def quanv(image):
    """Aplica o circuito quântico em toda a imagem."""
    if image.ndim == 2:  # Caso não tenha canal, adiciona dimensão
        image = image.unsqueeze(-1)
    out = np.zeros((14, 14, 4))

    for j in range(0, 28, 2):
        for k in range(0, 28, 2):
            q_results = circuit(
                [
                    image[j, k, 0],
                    image[j, k + 1, 0],
                    image[j + 1, k, 0],
                    image[j + 1, k + 1, 0]
                ]
            )
            for c in range(4):
                out[j // 2, k // 2, c] = q_results[c]
    return out

# Classe do modelo corrigida
class QuantumModel(nn.Module):
    def __init__(self):
        super(QuantumModel, self).__init__()
        self.flatten = nn.Flatten()
        self.fc = nn.Linear(14 * 14 * 4, 10) 

    def forward(self, x):
        # Aplica a convolução quântica
        batch_size = x.size(0)
        x = x.permute(0, 2, 3, 1) 
        x = x.detach().cpu().numpy() 
        x = np.array([quanv(img) for img in x]) 
        x = torch.as_tensor(x, dtype=torch.float32, device=device) 
        x = self.flatten(x)
        x = self.fc(x)
        return F.log_softmax(x, dim=1)


model = QuantumModel().to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
criterion = nn.CrossEntropyLoss().to(device)

for images, labels in dataloader_train:
    images, labels = images.squeeze(1).to(device), labels.squeeze().to(device)  
    optimizer.zero_grad()

    output = model(images)
    loss = criterion(output, labels)
    loss.backward()
    optimizer.step()
    print(f"Loss: {loss.item()}")




Loss: 2.598928213119507
Loss: 2.138389825820923


KeyboardInterrupt: 